# Importing Required Packages

In [9]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Function to scrape job postings from a single page

In [10]:
def scrape_page(driver):
    job_lst = []
    job_posts = driver.find_elements(By.CSS_SELECTOR, 'li')

    for job in job_posts:
        try:
            job_title_element = job.find_element(By.CSS_SELECTOR, '.base-search-card__title')
            
            job_title = job_title_element.text

            company_name = job.find_element(By.CSS_SELECTOR, '.base-search-card__subtitle a').text

            location = job.find_element(By.CSS_SELECTOR, '.job-search-card__location').text

            description = job.find_element(By.CSS_SELECTOR, '.job-posting-benefits__text').text

            posting_date = job.find_element(By.CSS_SELECTOR, 'time').text

            job_lst.append({
                'title': job_title,
                'company': company_name,
                'location': location,
                'date': posting_date,
                'description': description
            })

        except Exception as e:
            print(f"An error occurred: {e}")
    
    return job_lst

# Set up the Selenium WebDriver

In [11]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the job listings page

In [12]:
url = 'https://www.linkedin.com/jobs/search?keywords=Information%20Technology&location=India&geoId=102713980&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'  # Example URL, adjust as necessary

# Open the page

In [13]:
driver.get(url)
time.sleep(100)  # Wait for the JavaScript to render

all_job_lst = []
target_job_count = 1000  # Target number of jobs

# Scrape multiple pages

In [14]:
while len(all_job_lst) < target_job_count:
    all_job_lst.extend(scrape_page(driver))
    
    try:
        # Find and click the 'Next' button to go to the next page
        next_button = driver.find_element(By.XPATH, '//button[@aria-label="Next"]')  # Adjust the XPath as necessary
        if next_button:
            next_button.click()
            time.sleep(5)  # Wait for the next page to load
        else:
            break  # No 'Next' button found, end of pages
    except Exception as e:
        print(f"No more pages or error encountered: {e}")
        break

An error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":".base-search-card__title"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0052B793+45827]
	(No symbol) [0x004BDB74]
	(No symbol) [0x003B150F]
	(No symbol) [0x003F20BC]
	(No symbol) [0x003F216B]
	(No symbol) [0x003E8161]
	(No symbol) [0x00412E44]
	(No symbol) [0x003E8075]
	(No symbol) [0x00413094]
	(No symbol) [0x0042C034]
	(No symbol) [0x00412B96]
	(No symbol) [0x003E6998]
	(No symbol) [0x003E751D]
	GetHandleVerifier [0x007E43C3+2899763]
	GetHandleVerifier [0x008377ED+3240797]
	GetHandleVerifier [0x005B1264+593364]
	GetHandleVerifier [0x005B818C+621820]
	(No symbol) [0x004C6F54]
	(No symbol) [0x004C3658]
	(No symbol) [0x004C37F7]
	(No symbol) [0x004B58AE]
	BaseThreadInitThunk [0x75EC7BA9+25]
	RtlInitializeExcep

# Close the driver

In [15]:
driver.quit()

# Write the results to a CSV file

In [16]:
csv_file = 'job.csv'
keys = all_job_lst[0].keys()

with open(csv_file, 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_job_lst)

print(f"Collected {len(all_job_lst)} job postings.")
print(f"Job postings have been written to {csv_file}")

Collected 336 job postings.
Job postings have been written to job.csv
